# Model Performance with 95% Confidence Interval (Grouped by Q1 / Q2)

This notebook computes model performance (RMSE ± 95% CI) for Prophet, ARIMA, LSTM, and Random Forest, grouped by queries Q1 and Q2, for CPU, LatencyMs, and LogicalReads. The results are formatted for easy inclusion in a table like Table 1 in your paper.

In [ ]:
# !pip install numpy pandas scikit-learn xgboost statsmodels prophet tensorflow

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [3]:
csv_file = 'SimulatedQueryMetrics.csv'
df = pd.read_csv(csv_file)
df['MetricDate'] = pd.to_datetime(df['MetricDate'])
df = df.sort_values(['QueryName', 'MetricDate', 'QueryVariant']).reset_index(drop=True)
print('Columns:', df.columns.tolist())

Columns: ['SimDay', 'SimHour', 'MetricDate', 'QueryName', 'QueryVariant', 'CPU', 'LatencyMs', 'LogicalReads', 'PlanRegression']


## Utility: Lag Feature Builder, Data Splitting

In [5]:
def create_lag_features(df, lags=7, val_col='y'):
    df = df.copy()
    for lag in range(1, lags+1):
        df[f'lag_{lag}'] = df[val_col].shift(lag)
    df = df.dropna().reset_index(drop=True)
    return df

## Cross-Validation Function for 95% CI (Expanding Window Split)
We use 5 splits for each Query/Metric/Model, as in the example.

In [7]:
def expanding_window_cv(df, n_splits=5, lags=7):
    indices = np.array_split(np.arange(len(df)), n_splits)
    splits = []
    for i in range(1, n_splits):
        train_idx = np.concatenate(indices[:i])
        test_idx = indices[i]
        splits.append((train_idx, test_idx))
    return splits

## Main: Compute RMSE ± 95% CI for Each Model/Query/Metric

In [15]:
from xgboost import XGBRegressor
models = ['Prophet', 'ARIMA', 'LSTM', 'Random Forest', 'XGBoost']
metrics = ['CPU', 'LatencyMs', 'LogicalReads']
queries = ['Q1', 'Q2']
lags = 7
n_splits = 5
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

def get_ci95(arr):
    arr = np.array(arr)
    mean = arr.mean()
    se = arr.std(ddof=1) / np.sqrt(len(arr))
    ci = 1.96 * se
    return mean, ci

results = []
for query in queries:
    for metric in metrics:
        dfx = df[df['QueryName']==query].copy().sort_values(['MetricDate','QueryVariant'])
        dfx = dfx[['MetricDate', metric, 'QueryVariant','PlanRegression']].rename(columns={metric:'y','MetricDate':'ds'})
        dfx['y'] = dfx['y'].fillna(method='ffill').fillna(method='bfill').interpolate()
        dfx = create_lag_features(dfx, lags=lags, val_col='y')
        splits = expanding_window_cv(dfx, n_splits=n_splits, lags=lags)
        model_rmses = {m:[] for m in models}
        for train_idx, test_idx in splits:
            train_df = dfx.iloc[train_idx]
            test_df = dfx.iloc[test_idx]
            X_train = train_df[[f'lag_{i}' for i in range(1, lags+1)]]
            y_train = train_df['y']
            X_test = test_df[[f'lag_{i}' for i in range(1, lags+1)]]
            y_test = test_df['y']

            # Prophet
            prophet_train = train_df[['ds','y']]
            prophet_test = test_df[['ds','y']]
            pm = Prophet()
            pm.fit(prophet_train)
            pf = pm.predict(prophet_test[['ds']])
            prophet_pred = pf['yhat'].values
            model_rmses['Prophet'].append(mean_squared_error(y_test, prophet_pred, squared=False))

            # ARIMA
            arima_train = train_df['y']
            arima_test = test_df['y']
            am = ARIMA(arima_train, order=(lags,0,0)).fit()
            arima_pred = am.forecast(steps=len(arima_test))
            model_rmses['ARIMA'].append(mean_squared_error(arima_test, arima_pred, squared=False))

            # LSTM
            X_train_lstm = X_train.values.reshape((-1, lags, 1))
            X_test_lstm = X_test.values.reshape((-1, lags, 1))
            lstm_model = Sequential()
            lstm_model.add(LSTM(32, input_shape=(lags, 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mse')
            lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=0)
            lstm_pred = lstm_model.predict(X_test_lstm).flatten()
            model_rmses['LSTM'].append(mean_squared_error(y_test, lstm_pred, squared=False))

            # Random Forest
            rf = RandomForestRegressor(n_estimators=100, random_state=seed)
            rf.fit(X_train, y_train)
            rf_pred = rf.predict(X_test)
            model_rmses['Random Forest'].append(mean_squared_error(y_test, rf_pred, squared=False))

            # xgboost
            xgb = XGBRegressor(n_estimators=100, random_state=seed, verbosity=0)
            xgb.fit(X_train, y_train)
            xgb_pred = xgb.predict(X_test)
            model_rmses['XGBoost'].append(mean_squared_error(y_test, rf_pred, squared=False))

        # Summarize with 95% CI
        for m in models:
            mean, ci = get_ci95(model_rmses[m])
            results.append({
                'Query': query,
                'Metric': metric,
                'Model': m,
                'RMSE (mean)': mean,
                'CI95': ci,
                'Formatted': f"{mean:.2f} ± {ci:.2f}"
            })

06:59:46 - cmdstanpy - INFO - Chain [1] start processing
06:59:46 - cmdstanpy - INFO - Chain [1] done processing


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


06:59:56 - cmdstanpy - INFO - Chain [1] start processing
06:59:56 - cmdstanpy - INFO - Chain [1] done processing


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


07:00:09 - cmdstanpy - INFO - Chain [1] start processing
07:00:09 - cmdstanpy - INFO - Chain [1] done processing


...

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


07:05:28 - cmdstanpy - INFO - Chain [1] start processing
07:05:28 - cmdstanpy - INFO - Chain [1] done processing


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


07:05:42 - cmdstanpy - INFO - Chain [1] start processing
07:05:42 - cmdstanpy - INFO - Chain [1] done processing


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step


## Format Results Table: Grouped by Q1 / Q2

In [17]:
perf_df = pd.DataFrame(results)
# Pivot to match Table 1: Models as index, (Query, Metric) as columns, showing mean ± CI
tbl1 = perf_df.pivot(index='Model', columns=['Query','Metric'], values='Formatted')
tbl1 = tbl1[['Q1','Q2']] # ensure correct query order
display(tbl1)
tbl1.to_csv('model_performance_Q1_Q2_CI.csv')

Query                     Q1                                            Q2  \
Metric                   CPU       LatencyMs   LogicalReads            CPU   
Model                                                                        
ARIMA           17.54 ± 4.45   40.06 ± 12.68   26.09 ± 4.16   17.34 ± 4.52   
LSTM            45.55 ± 9.76  200.44 ± 19.22  158.63 ± 9.54  49.88 ± 10.58   
Prophet        29.49 ± 11.34   59.70 ± 16.17  38.50 ± 13.17   28.25 ± 9.86   
Random Forest    8.97 ± 3.72   21.46 ± 10.51   14.92 ± 3.98    8.13 ± 2.59   
XGBoost          8.97 ± 3.72   21.46 ± 10.51   14.92 ± 3.98    8.13 ± 2.59   

Query                                         
Metric              LatencyMs   LogicalReads  
Model                                         
ARIMA           39.27 ± 12.33   26.11 ± 4.31  
LSTM           208.28 ± 15.72  166.27 ± 7.47  
Prophet         57.11 ± 14.71  39.82 ± 13.88  
Random Forest   20.97 ± 10.18   13.31 ± 3.21  
XGBoost         20.97 ± 10.18   13.31 ± 3.21

## (Optional) Show Table as LaTeX

In [19]:
from tabulate import tabulate
print(tabulate(tbl1, headers='keys', tablefmt='latex'))

\begin{tabular}{lllllll}
\hline
 Model         & ('Q1', 'CPU')   & ('Q1', 'LatencyMs')   & ('Q1', 'LogicalReads')   & ('Q2', 'CPU')   & ('Q2', 'LatencyMs')   & ('Q2', 'LogicalReads')   \\
\hline
 ARIMA         & 17.54 ± 4.45    & 40.06 ± 12.68         & 26.09 ± 4.16             & 17.34 ± 4.52    & 39.27 ± 12.33         & 26.11 ± 4.31             \\
 LSTM          & 45.55 ± 9.76    & 200.44 ± 19.22        & 158.63 ± 9.54            & 49.88 ± 10.58   & 208.28 ± 15.72        & 166.27 ± 7.47            \\
 Prophet       & 29.49 ± 11.34   & 59.70 ± 16.17         & 38.50 ± 13.17            & 28.25 ± 9.86    & 57.11 ± 14.71         & 39.82 ± 13.88            \\
 Random Forest & 8.97 ± 3.72     & 21.46 ± 10.51         & 14.92 ± 3.98             & 8.13 ± 2.59     & 20.97 ± 10.18         & 13.31 ± 3.21             \\
 XGBoost       & 8.97 ± 3.72     & 21.46 ± 10.51         & 14.92 ± 3.98             & 8.13 ± 2.59     & 20.97 ± 10.18         & 13.31 ± 3.21             \\
\hline
\end{tabular}
